In [ ]:
!pip install torch==1.8.1
!pip install torchtext==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
from time import time
import torch
import torch.nn as nn
import torch.utils.data
import time
import torch.nn.functional as F
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import TabularDataset, BucketIterator
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv("/content/NewsCategorizer.csv")
df.head()

,category,headline,links,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           50000 non-null  object
 1   headline           50000 non-null  object
 2   links              50000 non-null  object
 3   short_description  50000 non-null  object
 4   keywords           47332 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [ ]:
df.category.unique()

array(['WELLNESS', 'POLITICS', 'ENTERTAINMENT', 'TRAVEL',
       'STYLE & BEAUTY', 'PARENTING', 'FOOD & DRINK', 'WORLD NEWS',
       'BUSINESS', 'SPORTS'], dtype=object)

In [ ]:
df.drop('links', axis = 1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           50000 non-null  object
 1   headline           50000 non-null  object
 2   short_description  50000 non-null  object
 3   keywords           47332 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [ ]:
df.isnull().sum()

category                0
headline                0
short_description       0
keywords             2668
dtype: int64

In [ ]:
df1 = df.dropna(axis = 0)
print(f"Before dropping all NaN values: {df.shape}")
print(f"After dropping all NaN values: {df1.shape}")

Before dropping all NaN values: (50000, 4)
After dropping all NaN values: (47332, 4)


In [ ]:
object_dtypes = df.select_dtypes(include = 'object')
object_dtypes.head()

,category,headline,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made","If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,"First, the bad news: Soda bread, corned beef a...",green-superfoods


In [ ]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

#stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#lemmatizer
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def text_tokenization(text):
    text = text.replace("n't", ' not')
    text = text.replace("'ve", ' have')
    text = text.replace("'m", ' am')
    text = text.replace("’s", ' ')
    text = text.replace("'s", ' ')
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub("[..,,ㆍ\'\"’‘”“!?\\‘|\(\)\[\]\<\>`\'◇…]", "", text)
    text = re.sub(r'[^a-zA-z]+', " ", text) # only english remain
    text = re.sub(" +", " ", text) #remove multi-space
    text = text.lower()
    return " ".join([stemmer.stem(word) for word in str(text).split()
                     if word not in stop_words])

def cnn_tokenization(text):
    text = text.replace("n't", ' not')
    text = text.replace("'ve", ' have')
    text = text.replace("'m", ' am')
    text = text.replace("’s", ' ')
    text = text.replace("'s", ' ')
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub("[..,,ㆍ\'\"’‘”“!?\\‘|\(\)\[\]\<\>`\'◇…]", "", text)
    text = re.sub(r'[^a-zA-z]+', " ", text) # only english remain
    text = text.lower()
    return [stemmer.stem(word) for word in str(text).split()
                     if word not in stop_words]

def only_lemmatization(text):
    text = text.replace("n't", ' not')
    text = text.replace("'ve", ' have')
    text = text.replace("'m", ' am')
    text = text.replace("’s", ' ')
    text = text.replace("'s", ' ')
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub("[..,,ㆍ\'\"’‘”“!?\\‘|\(\)\[\]\<\>`\'◇…]", "", text)
    text = re.sub(r'[^a-zA-z]+', " ", text) # only english remain
    text = re.sub(" +", " ", text) #remove multi-space
    text = text.lower()
    text = " ".join([lemma.lemmatize(word, pos='v') for word in text.split() if word not in stop_words])
    return text

In [ ]:
df_final = df[["category",  "keywords"]]
df_final["cleaned_headline"] = df["headline"].apply(text_tokenization)
df_final["cleaned_description"] = df["short_description"].apply(text_tokenization)

In [ ]:
df_final.head(2)

,category,keywords,cleaned_headline,cleaned_description
0,WELLNESS,running-lessons,mile day lesson learn,rest part train confirm sort alreadi knew buil...
1,WELLNESS,talking-to-yourself-crazy,talk crazi crazi help,think talk tool coach challeng narrat experi c...


In [ ]:
df_wellness = df_final[df_final["category"]==0]

df_politics = df_final[df_final["category"]==1]
df_entertainment = df_final[df_final["category"]==2]
df_travel = df_final[df_final["category"]==3]
df_stylenbeauty = df_final[df_final["category"]==4]
df_parenting = df_final[df_final["category"]==5]
df_foodndrink = df_final[df_final["category"]==6]
df_worldnews = df_final[df_final["category"]==7]
df_business = df_final[df_final["category"]==8]
df_sports = df_final[df_final["category"]==9]

In [ ]:
df["cleaned"] = df["short_description"].apply(text_tokenization)
df.head(2)

,category,headline,short_description,keywords,cleaned
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,Resting is part of training. I've confirmed wh...,running-lessons,rest part train confirm sort alreadi knew buil...
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy,think talk tool coach challeng narrat experi c...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer().fit(df['cleaned'])

In [ ]:
category_bow = bow_transformer.transform(df['cleaned'])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(category_bow)

In [ ]:
category_tfidf=tfidf_transformer.transform(category_bow)
print(category_tfidf.shape)

(50000, 29051)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

category_detect_model = MultinomialNB().fit(category_tfidf, df['category'])
NB_predictions = category_detect_model.predict(category_tfidf)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(df['category'], NB_predictions))
print(confusion_matrix(df['category'], NB_predictions))

                precision    recall  f1-score   support

      BUSINESS       0.73      0.78      0.75      5000
 ENTERTAINMENT       0.83      0.62      0.71      5000
  FOOD & DRINK       0.82      0.80      0.81      5000
     PARENTING       0.63      0.81      0.71      5000
      POLITICS       0.79      0.70      0.74      5000
        SPORTS       0.88      0.76      0.82      5000
STYLE & BEAUTY       0.82      0.78      0.80      5000
        TRAVEL       0.77      0.79      0.78      5000
      WELLNESS       0.65      0.81      0.72      5000
    WORLD NEWS       0.82      0.79      0.80      5000

      accuracy                           0.76     50000
     macro avg       0.77      0.76      0.76     50000
  weighted avg       0.77      0.76      0.76     50000

[[3902   40   54  218  191   44   43  104  321   83]
 [ 158 3122  147  460  149  140  303  177  257   87]
 [  86   45 3986  254   22   37   97  213  240   20]
 [ 110   82   85 4028   62   29   90   81  405   28]
 

In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

category_detect_model2 = LogisticRegression(solver='liblinear').fit(category_tfidf, df['category'])
LR_predictions = category_detect_model2.predict(category_tfidf)
print(classification_report(df['category'], LR_predictions))
print(confusion_matrix(df['category'], LR_predictions))

                precision    recall  f1-score   support

      BUSINESS       0.78      0.79      0.78      5000
 ENTERTAINMENT       0.73      0.75      0.74      5000
  FOOD & DRINK       0.80      0.84      0.82      5000
     PARENTING       0.78      0.76      0.77      5000
      POLITICS       0.79      0.71      0.75      5000
        SPORTS       0.82      0.86      0.84      5000
STYLE & BEAUTY       0.83      0.80      0.81      5000
        TRAVEL       0.80      0.78      0.79      5000
      WELLNESS       0.74      0.79      0.76      5000
    WORLD NEWS       0.80      0.81      0.80      5000

      accuracy                           0.79     50000
     macro avg       0.79      0.79      0.79     50000
  weighted avg       0.79      0.79      0.79     50000

[[3931  102   73  119  216   78   50  106  220  105]
 [  88 3751  145  172  105  216  212  103  114   94]
 [  75  121 4188  103   20   84  109  155  117   28]
 [ 119  169  132 3779   67   81  123   99  390   41]
 

In [ ]:
X = df.iloc[:, [-1]].values
y = df.iloc[:, 0].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,0] = le.fit_transform(X[:,0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
knn_cm = confusion_matrix(y_test, y_pred)
knn_ac = accuracy_score(y_test,y_pred)

In [ ]:
knn_ac

0.2875333333333333

In [ ]:

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
# Predict Output
rf_predicted = random_forest.predict(X_test)

random_forest_score = round(random_forest.score(X_train, y_train) * 100, 2)
random_forest_score_test = round(random_forest.score(X_test, y_test) * 100, 2)
print('Random Forest Score: \n', random_forest_score)
print('Random Forest Test Score: \n', random_forest_score_test)
print('Accuracy: \n', accuracy_score(y_test,rf_predicted))
print(confusion_matrix(y_test,rf_predicted))
print(classification_report(y_test,rf_predicted))

Random Forest Score: 
 99.82
Random Forest Test Score: 
 32.85
Accuracy: 
 0.3284666666666667
[[ 895   54   75   78   71   47   49   60   66   61]
 [  98  367  138  122  119  114  173  127  123  127]
 [  88  139  379  152  107   72  142  181  177   73]
 [  98  121  158  321  121   67  140  134  189  115]
 [ 126  136   95  106  418   91   87  122  127  211]
 [  30   69   39   42   74 1070   37   58   65   43]
 [  87  168  147  141   95   81  416  163  112  111]
 [  93  140  183  173  123   69  151  278  150  152]
 [ 117  135  181  191  118   84  126  127  321   98]
 [ 125  130   81   90  190   91   71  135  110  462]]
                precision    recall  f1-score   support

      BUSINESS       0.51      0.61      0.56      1456
 ENTERTAINMENT       0.25      0.24      0.25      1508
  FOOD & DRINK       0.26      0.25      0.25      1510
     PARENTING       0.23      0.22      0.22      1464
      POLITICS       0.29      0.28      0.28      1519
        SPORTS       0.60      0.70   